# Geospatial Data Analysis

In [ ]:
import geopandas
import re
import pandas as pd
import folium
import requests
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

## 1. Master Plan 2019  

In this section, we explore the geospatial data for URA's Master Plan 2019 - specifically the subzones data as well as the land use data. We will be using this as an opportunity to learn how to read, clean, analyse, and visualise geospatial data relating to areas (polygons).

### 1.1 MP19 Planning Area and Subzones

We start by exploring the planning areas and subzones in the Master Plan 2019. Download the GeoJSON file from `data.gov.sg` [here](https://beta.data.gov.sg/datasets/d_8594ae9ff96d0c708bc2af633048edfb/view).

In [ ]:
MP19_Subzone = geopandas.read_file("MasterPlan2019SubzoneBoundaryNoSeaGEOJSON.geojson")

In [ ]:
# Look at the structure of the dataset
MP19_Subzone.head()

In [ ]:
# Observe how messy the description tag is
MP19_Subzone['Description'][0]

In [ ]:
# Write a simple function to extract the region, planning area, and subzone from the HTML string
def extract_subzone_description(string):
    results = re.findall('<td>([A-Z0-9&/\-()\' ]*)</td>', string)
    if len(results) >= 2:
        return {
            'Subzone': results[1],
            'Planning Area': results[4],
            'Region': results[7]
        }
    else:
        raise AssertionError("No subzone data detected")

In [ ]:
# Check that it works
extract_subzone_description(MP19_Subzone['Description'][0])

In [ ]:
MP19_Subzone_Additional = MP19_Subzone.apply(lambda row: extract_subzone_description(row.Description), axis = 1, result_type = 'expand')
MP19_Subzone = pd.concat([MP19_Subzone, MP19_Subzone_Additional], axis = 1)

In [ ]:
# Check if our function ran correctly
MP19_Subzone.head()

<span style="background-color: #FFFF00">**Exercise:** How many planning areas and subzones are there in Singapore?</span>

In [ ]:
# Fill this part in yourself (two lines only)


<span style="background-color: #FFFF00">**Exercise:** Which region has the most subzones?</span>

In [ ]:
# Fill this part in yourself (one line only)


Let's visualise this using Folium so we can see the different subzones in Singapore. We start with loading the Singapore basemap.

In [ ]:
MP19_Subzone_Folium = folium.Map(location = (1.359394, 103.814301), zoom_start = 12)
MP19_Subzone_Folium

Let's layer on the subzone data that we have using the GeoJSON file.

In [ ]:
folium.GeoJson(MP19_Subzone, name = "MP19 Subzones").add_to(MP19_Subzone_Folium)
MP19_Subzone_Folium

This looks great! But it's hard to understand which subzone we are looking at. Let's add a tooltip so we can automatically see what subzone our cursor is hovering over. We'll also colour the planning area so we can see where the respec

In [ ]:
# Set up the Tableau 20 colour mapping
from matplotlib import colormaps, colors
planning_area_cmap = {}
for i, planningarea in enumerate(MP19_Subzone['Planning Area']):
    planning_area_cmap[planningarea] = colors.rgb2hex(colormaps['tab20'].colors[i%20])

In [ ]:
MP19_Subzone_Folium = folium.Map(location = (1.359394, 103.814301), zoom_start = 12)
MP19_Subzone_Tooltip = folium.GeoJsonTooltip(
    fields = ["Subzone", "Planning Area", "Region"],
    localize = True,
    sticky = False,
    labels = True,
    style = """
        background-color: #F0EFEF;
        border: 0.5px solid black;
        border-radius: 3px;
        box-shadow: 2px;
    """,
    max_width=800,
)
folium.GeoJson(MP19_Subzone, 
               name = "MP19 Subzones",
               tooltip = MP19_Subzone_Tooltip,
               style_function = lambda x: {
                   "fillColor": planning_area_cmap[x["properties"]["Planning Area"]],
                   'color': '#444444',
                   'weight': 0.7,
                   "fillOpacity": 0.5,
               }).add_to(MP19_Subzone_Folium)
MP19_Subzone_Folium

<span style="background-color: #FFFF00">**Class Discussion:** Take a close look at the map above, and think about the following points: </span> 
* Do the planning areas correspond to what you had expected (especially certain neighbourhoods)?
* Is there anything that surprised you? Name an example of a subzone / planning area which was unexpected for you.
* How about the regions? Do the regions make sense to you?

### 1.2 MP19 Land Use

Next, we look at the land use layer to get a better understanding of how Singapore's land is being used. Download the GeoJSON file from `data.gov.sg` [here](https://beta.data.gov.sg/datasets/d_90d86daa5bfaa371668b84fa5f01424f/view).



In [ ]:
MP19_LandUse = geopandas.read_file("MasterPlan2019LandUselayer.geojson")

In [ ]:
# Write a simple function to extract the land use description from the HTML string
def extract_landuse_description(string):
    results = re.findall('<td>([A-Z0-9&/\- ]*)</td>', string)
    if len(results) >= 2:
        return results[0]
    else:
        raise AssertionError("No land use information detected")

In [ ]:
MP19_LandUse['LandUse'] = MP19_LandUse['Description'].apply(extract_landuse_description)

In [ ]:
# Check if our function ran correctly
MP19_LandUse.head()

<span style="background-color: #FFFF00">**Exercise:** How many different land use categories are there? What is the most commmon land use for land parcels in Singapore?</span>

In [ ]:
# Fill this part in yourself (two lines only)


<span style="background-color: #FFFF00">**Exercise:** Generate a Folium map of land use parcels in Singapore, with each land use type coloured differently.</span>

In [ ]:
# Run this to clean up some invalid polygons
MP19_LandUse['geometry'] = MP19_LandUse['geometry'].buffer(0)

# Run this to aggregate the large dataset into something more manageable for visualisation
MP19_LandUse_Agg = MP19_LandUse[['LandUse', 'geometry']].dissolve(by = 'LandUse').reset_index()

In [ ]:
MP19_LandUse_Agg.head()

In [ ]:
# Fill this part in yourself


<span style="background-color: #FFFF00">**Class Discussion:** What do you observe about how land use is distributed across Singapore?</span>
* What do you think are the differences between `Business 1` and `Business 2` areas? How does this show up in where they are found in Singapore?
* There are some mixed-use land parcels (like residential and commmercial). Where are they usually found at, and why do you think that is the case?
* Where are the largest reserve sites at? What do you think they will be used for?
* How would you improve how this map is visualised?

<span style="background-color: #FFFF00">**Exercise:** Calculate the area (in kilometre squared) for each land use category, and identify the category with the largest land use in Singapore.</span>

In [ ]:
# To do this correctly, you'll need to reproject the CRS from a geographic CRS to a projected CRS
MP19_LandUse_Agg_reproj = MP19_LandUse_Agg.to_crs(3857)

In [ ]:
# Fill this in (two lines only)


## 2) HDB Flats

In this section, we explore the HDB Property Information dataset. This is not strictly a geospatial dataset, but it provides the address which we can geocode to get the latitude and longitude, with which we can do further geospatial analysis. This section will show you how to geocode data and combine it with other geospatial data types using geospatial operations.

In [ ]:
hdb_property_info = pd.read_csv('HDBPropertyInformation.csv')

In [ ]:
hdb_property_info

<span style="background-color: #FFFF00">**Exercise:** How many HDB flats are there in Singapore? How many HDB blocks are there?</span>

In [ ]:
# Fill this part in yourself (two lines only)


<span style="background-color: #FFFF00">**Exercise:** Plot a line chart showing how many HDB flats were built annually since the beginning till today.</span>

In [ ]:
# Fill this part in yourself (one line only)


This is fairly basic data analysis, so let's turn it into a geospatial dataset. We need to convert the address into a point by finding its coordinates (latitude and longitude). You can do this for free by using Singapore's OneMap API - you can find the documentation [here](https://www.onemap.gov.sg/apidocs/apidocs).

For the `Search` endpoint, you don't need any authentication. For the other endpoints, you need to pass in an API key as well, which you can sign up for [here](https://www.onemap.gov.sg/apidocs/register).

<span style="background-color: #FFFF00">**Exercise:** Write a function that takes the address as input, calls the OneMap Search API, and returns the latitude and longitude.</span>

In [ ]:
# Fill in the function here
def call_onemap_api(address):
    
    return {
        'Longitude': # Fill this in,
        'Latitude': # Fill this in
    }

In [ ]:
# Test your function here

# It should return {'Longitude': '103.781566663057', 'Latitude': '1.30591387643498'}
print(call_onemap_api('26 DOVER CRES'))

# It should return {'Longitude': '103.884012115114', 'Latitude': '1.30129028933217'}
print(call_onemap_api('6 JLN BATU'))

# It should return {'Longitude': '103.847769289898', 'Latitude': '1.33862553170269'}
print(call_onemap_api('97 LOR 3 TOA PAYOH'))

In [ ]:
hdb_property_info['Address'] = hdb_property_info['blk_no'] + " " + hdb_property_info['street']

In [ ]:
# This will take a while as we are calling the API over 12k times
#geocode_output = []
for i, address in tqdm(enumerate(hdb_property_info['Address'])):

    # If the HDB property has already been geocoded before, we check if it failed precisely
    if i < len(geocode_output):

        # If it failed previously, we retry it
        if geocode_output[i]['Longitude'] is None:
            try:
                geocode_output[i] = call_onemap_api(address)
            except AssertionError:
                geocode_output[i] = {'Longitude': None, 'Latitude': None}

    # If it hasn't been geocoded before, we geocode it now
    else:
        try:
            geocode_output.append(call_onemap_api(address))
        except AssertionError:
            geocode_output.append({'Longitude': None, 'Latitude': None})

In [ ]:
hdb_property_info = pd.concat([hdb_property_info, pd.DataFrame(geocode_output)], axis = 1)

The full dataset has been done for you in the interests of time, so just read in the data using the command below.

In [ ]:
#hdb_property_info.to_csv("HDBPropertyInformation_LatLon.csv", index = False)
hdb_property_info = pd.read_csv("HDBPropertyInformation_LatLon.csv")

In [ ]:
# Convert it into a GeoDataFrame
hdb_property_info_gdf = geopandas.GeoDataFrame(hdb_property_info, 
                                               geometry = geopandas.points_from_xy(hdb_property_info.Longitude, hdb_property_info.Latitude),
                                               crs = "EPSG:4326")

In [ ]:
hdb_property_info_gdf.head()

In [ ]:
hdb_flat_info_gdf = hdb_property_info_gdf[hdb_property_info_gdf['residential'] == 'Y'].reset_index(drop = True)

<span style="background-color: #FFFF00">**Class Discussion:** What is the main difference between the GeoDataFrame and the original dataframe that you read in from the CSV file? Why is this significant?</span>

With the geospatial data loaded, we can now plot our HDB flats data. Let's do a simpler static plot this time, and overlay it with the Master Plan 2019 subzones data so we can roughly tell where the HDB flats are.

In [ ]:
ax = MP19_Subzone.plot(color = "white", edgecolor = "black", figsize = (12, 9))
hdb_flat_info_gdf.plot(ax = ax, color = 'blue', markersize = 0.5)
plt.show()

In [ ]:
ax = MP19_Subzone.plot(color = "white", edgecolor = "black", figsize = (12, 9))
hdb_flat_info_gdf.plot(ax = ax, 
                       markersize = 0.5,
                       column = hdb_flat_info_gdf['year_completed'],
                       legend = True)
plt.show()

<span style="background-color: #FFFF00">**Class Discussion:** Just looking at the plots above, what do you observe from these static geospatial plots?</span>
* Where are the older flats concentrated at? How about the newer flats? Is this similar to what you had expected before plotting out the data?
* In which subzones are the concentration of HDB flats among the highest? How about residential subzones with few HDB flats - which ones do you think would make the top of the list?

<span style="background-color: #FFFF00">**Exercise:** Find out which subzone has the highest average age of HDB flats. Then find out which subzone has the lowest average age of HDB flats.</span>


In [ ]:
# Fill it in here


<span style="background-color: #FFFF00">**Exercise:** Which subzone has the highest concentration of HDB flats (total number of flats divided by km^2)?</span>

In [ ]:
# Fill this in


<span style="background-color: #FFFF00">**Class Discussion:** Do you spot anything odd with the subzones with the lowest flat densities?</span>

## 3) LTA Public Transport

In this section, we explore LTA's public transportation data, specifically the Bus Stops API. You will learn how to call APIs (if you don't already know), convert it into a GeoDataFrame, visualise it, and compute distances between two points.

In [ ]:
# Note that you need to sign up for your own account and API key first
# Sign up here: https://datamall.lta.gov.sg/content/datamall/en/request-for-api.html
lta_api_key = ""

<span style="background-color: #FFFF00">**Exercise:** Write a function that calls the Bus Stops API and saves all the information into a Pandas dataframe. Then convert it into a GeoPandas dataframe.</span>   
*Hint: Read through the API documentation [here](https://datamall.lta.gov.sg/content/dam/datamall/datasets/LTA_DataMall_API_User_Guide.pdf)*

In [ ]:
def call_bus_stop_api(lta_api_key):

    # fill in the function here

    return df

In [ ]:
# Test your function here
bus_stops = call_bus_stop_api(lta_api_key)

In [ ]:
# You should see the shape as (5134, 5)
bus_stops.shape

In [ ]:
bus_stops.head()

In [ ]:
# Convert it into a GeoDataFrame (one line only)


In [ ]:
# Check that it ran correctly
ax = MP19_Subzone.plot(color = "white", edgecolor = "black", figsize = (12, 9))
bus_stops_gdf.plot(ax = ax, color = 'red', markersize = 0.5)
plt.show()

<span style="background-color: #FFFF00">**Class Discussion:** What do you observe from the plot above? Are there any interesting observations that you can notice immediately?</span>
* Are there any bus stops that in surprising locations?
* Some subzones have a lot of bus stops, while some have very few bus stops. Can you identify which ones, and guess why that is the case?

<span style="background-color: #FFFF00">**Exercise:** Which subzone has the most number of bus stops? Which subzone has the least number (non-zero) of bus stops?</span>   


In [ ]:
# Fill in your code here


This looks a bit strange especially at the lower end, since Maxwell and Little India are right in the middle of town. Let's investigate this further by generating an interactive map so we can zoom into those two subzones and see what's happening.

<span style="background-color: #FFFF00">**Exercise:** Generate a Folium map with all the bus stops overlaid over the Master Plan 2019 subzones. Make sure to include tooltips for each of the bus stops.</span>   

In [ ]:
# Fill this in


Now that we can see the distribution of bus stops over Singapore, how about its relation to HDB flats? We all like to have bus stops nearby, but which HDB flat is closest to a bus stop, and which HDB flat is furthest away?

<span style="background-color: #FFFF00">**Exercise:** Find which HDB flat is the closest to any bus stop, and indicate the distance in metres. Then find which HDB flat is furthest, and how far it is away.</span>   

In [ ]:
# Fill this in
